`triton_call` cannot (currently) be vmapped - so we must write a multi-headed causal self
attention kernel instead.

We can probably write a faster version of `FSingleHeadCausalSelfAttention.get_qKV` using
a Triton kernel:
  - Linear layer Triton impl from https://github.com/ELS-RD/kernl/blob/main/src/kernl/implementations/linear_layer.py
  - Use Triton to write back to the cache, rather than dynamic_update_slice
  - Fuse the layer norm into the linear layer. Compute layer norm in a single pass using
  https://github.com/ELS-RD/kernl/blob/main/src/kernl/implementations/layer_norm.py - then
  pass the mean and variance into the linear layer impl.

# Layer norm and linear layer

We implement a fused linear layer + embedding to q, k, v linear layer using Triton. Implement:

- baseline